In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('../data/Hemonc_new.csv')


df_demo = df.head()
df["pmids"][2]

'18955563, 20418244'

In [45]:
from datetime import datetime
import pandas as pd
import json
import requests
from Bio import Entrez
import re
from bs4 import BeautifulSoup

# Set your email for NCBI Entrez
Entrez.email = "yuxin102@mit.edu"

def fetch_pubmed_data(pubmed_id: str) -> dict:
    """
    Fetches author list for a given PubMed ID.
    
    Args:
        pubmed_id: The PubMed ID string.
        
    Returns:
        A dictionary containing 'authors' (pipe-separated string)
    """
    authors_str = ""
    
    try:
        # Fetch the PubMed record in XML format
        handle = Entrez.efetch(db="pubmed", id=pubmed_id, retmode="xml")
        records = Entrez.read(handle)
        handle.close()
        
        # Check if records were fetched and contain the expected structure
        if records and "PubmedArticle" in records:
            article = records["PubmedArticle"][0].get("MedlineCitation", {}).get("Article", {})
            
            # Extract the author list
            author_list = []
            if "AuthorList" in article:
                for author in article["AuthorList"]:
                    # Check if the author has separate first and last names
                    if "ForeName" in author and "LastName" in author:
                        full_name = f"{author['ForeName']} {author['LastName']}"
                    elif "CollectiveName" in author:
                        full_name = author["CollectiveName"]
                    else:
                        # Fallback if name parts are missing
                        full_name = author.get("LastName", "Name Not Available")
                    author_list.append(full_name)
            
            authors_str = "|".join(author_list)
    
    except Exception as e:
        print(f"Error fetching data for PubMed ID {pubmed_id}: {e}")
        return {"authors": ""}
    
    return {"authors": authors_str}

def find_nct_from_pubmed(pubmed_id: str) -> str:
    """
    Attempts to find an NCT ID from a PubMed ID.
    
    Args:
        pubmed_id: The PubMed ID string.
    
    Returns:
        NCT ID if found, empty string otherwise.
    """
    try:
        # Fetch the PubMed record using Entrez
        handle = Entrez.efetch(db="pubmed", id=pubmed_id, retmode="xml")
        data = handle.read()
        
        # Handle different return types - might be bytes or string
        if isinstance(data, bytes):
            data = data.decode('utf-8')
        
        handle.close()
        
        # Use regex to find NCT IDs in the XML data
        nct_pattern = re.compile(r'NCT\d{8}')
        matches = nct_pattern.findall(data)
        
        if matches:
            return matches[0]  # Return the first NCT ID found
        
        return ""
    
    except Exception as e:
        print(f"Error finding NCT ID for PubMed ID {pubmed_id}: {e}")
        return ""

def identify_drug_class(intervention_name: str) -> str:
    """
    Identifies the drug class from the intervention name.
    
    Args:
        intervention_name: The name of the intervention/drug.
    
    Returns:
        String indicating the drug class.
    """
    # Convert to lowercase for case-insensitive matching
    intervention_lower = intervention_name.lower()
    
    # Comprehensive drug classifications
    drug_classes = {
        # PD-1 Inhibitors
        "pd-1 inhibitors": ["pembrolizumab", "nivolumab", "cemiplimab", "dostarlimab", "keytruda", "opdivo"],
        
        # PD-L1 Inhibitors
        "pd-l1 inhibitors": ["atezolizumab", "avelumab", "durvalumab", "tecentriq", "bavencio", "imfinzi"],
        
        # CTLA-4 Inhibitors
        "ctla-4 inhibitors": ["ipilimumab", "tremelimumab", "yervoy"],
        
        # Anti-VEGF
        "anti-vegf": ["bevacizumab", "ramucirumab", "aflibercept", "avastin", "cyramza", "zaltrap"],
        
        # Tyrosine Kinase Inhibitors
        "tyrosine kinase inhibitors": ["imatinib", "sunitinib", "erlotinib", "gefitinib", "sorafenib", "gleevec", 
                                       "sutent", "tarceva", "iressa", "nexavar", "pazopanib", "votrient"],
        
        # EGFR Inhibitors
        "egfr inhibitors": ["erlotinib", "gefitinib", "osimertinib", "afatinib", "tarceva", "iressa", "tagrisso", "gilotrif"],
        
        # Platinum-based Chemotherapy
        "platinum-based chemotherapy": ["cisplatin", "carboplatin", "oxaliplatin"],
        
        # Taxanes
        "taxanes": ["paclitaxel", "docetaxel", "cabazitaxel", "taxol", "taxotere", "jevtana"],
        
        # Antimetabolites
        "antimetabolites": ["gemcitabine", "fluorouracil", "5-fu", "pemetrexed", "capecitabine", "gemzar", "alimta", "xeloda"],
        
        # Immunomodulatory Drugs
        "immunomodulatory drugs": ["thalidomide", "lenalidomide", "pomalidomide", "thalomid", "revlimid", "pomalyst"],
        
        # Corticosteroids
        "corticosteroids": ["dexamethasone", "prednisone", "prednisolone", "methylprednisolone", "decadron"],
        
        # BTK Inhibitors
        "btk inhibitors": ["ibrutinib", "acalabrutinib", "zanubrutinib", "imbruvica", "calquence", "brukinsa"],
        
        # CDK4/6 Inhibitors
        "cdk4/6 inhibitors": ["palbociclib", "ribociclib", "abemaciclib", "ibrance", "kisqali", "verzenio"],
        
        # PARP Inhibitors
        "parp inhibitors": ["olaparib", "rucaparib", "niraparib", "talazoparib", "lynparza", "rubraca", "zejula", "talzenna"],
        
        # mTOR Inhibitors
        "mtor inhibitors": ["everolimus", "temsirolimus", "afinitor", "torisel"],
        
        # Proteasome Inhibitors
        "proteasome inhibitors": ["bortezomib", "carfilzomib", "ixazomib", "velcade", "kyprolis", "ninlaro"],
        
        # Monoclonal Antibodies (General)
        "monoclonal antibodies": ["mab", "monoclonal", "antibody"]
    }
    
    # Check for drug class matches
    for class_name, drugs in drug_classes.items():
        for drug in drugs:
            if drug in intervention_lower:
                # Convert class name to title case for consistency
                return " ".join(word.capitalize() for word in class_name.split())
    
    # Check for class names directly in the intervention
    class_keywords = {
        "pd-1": "PD-1 Inhibitor",
        "pdl1": "PD-L1 Inhibitor",
        "pd-l1": "PD-L1 Inhibitor",
        "ctla-4": "CTLA-4 Inhibitor",
        "ctla4": "CTLA-4 Inhibitor",
        "vegf": "Anti-VEGF",
        "antivegf": "Anti-VEGF",
        "kinase": "Kinase Inhibitor",
        "tyrosine": "Tyrosine Kinase Inhibitor",
        "chemotherapy": "Chemotherapy",
        "chemo": "Chemotherapy",
        "immunotherapy": "Immunotherapy",
        "immune": "Immunotherapy",
        "egfr": "EGFR Inhibitor",
        "platinum": "Platinum-based Chemotherapy",
        "taxane": "Taxane",
        "antimetabolite": "Antimetabolite",
        "immunomodulatory": "Immunomodulatory Drug",
        "corticosteroid": "Corticosteroid",
        "steroid": "Corticosteroid",
        "btk": "BTK Inhibitor",
        "cdk": "CDK Inhibitor",
        "parp": "PARP Inhibitor",
        "mtor": "mTOR Inhibitor",
        "proteasome": "Proteasome Inhibitor"
    }
    
    for keyword, class_name in class_keywords.items():
        if keyword in intervention_lower:
            return class_name
    
    return "Unknown"

def extract_route_from_text(text: str) -> set:
    """
    Extracts drug administration routes from text.
    
    Args:
        text: The text to analyze.
    
    Returns:
        Set of identified routes.
    """
    routes = set()
    
    # Common route keywords and their formal names
    route_keywords = {
        "oral": "Oral",
        "by mouth": "Oral",
        "p.o.": "Oral",
        "intravenous": "Intravenous",
        "i.v.": "Intravenous",
        "iv infusion": "Intravenous",
        "iv push": "Intravenous",
        "subcutaneous": "Subcutaneous",
        "s.c.": "Subcutaneous",
        "subcu": "Subcutaneous",
        "intramuscular": "Intramuscular",
        "i.m.": "Intramuscular",
        "topical": "Topical",
        "intradermal": "Intradermal",
        "inhalation": "Inhalation",
        "intranasal": "Intranasal",
        "intrathecal": "Intrathecal",
        "rectal": "Rectal",
        "vaginal": "Vaginal"
    }
    
    # Check for route keywords in the text
    text_lower = text.lower()
    for keyword, formal_name in route_keywords.items():
        if keyword in text_lower:
            routes.add(formal_name)
    
    return routes

def fetch_clinicaltrials_gov_data(nct_id: str) -> dict:
    """
    Fetches data for a given NCT ID using multiple approaches.
    
    Args:
        nct_id: The NCT ID string (e.g., 'NCT00205751').
    
    Returns:
        Dictionary containing trial data including start_date, outcomes,
        drug routes, and drug classes.
    """
    result = {
        "start_date": "",
        "has_primary_outcome": "No",
        "has_secondary_outcome": "No",
        "drug_routes": set(),
        "drug_classes": set()
    }
    
    try:
        # Ensure the NCT ID is properly formatted
        if not nct_id.startswith("NCT"):
            nct_id = f"NCT{nct_id}"
        
        # APPROACH 1: Use the ClinicalTrials.gov V2 API
        api_url = f"https://clinicaltrials.gov/api/v2/studies/{nct_id}"
        response = requests.get(api_url)
        
        if response.status_code == 200:
            data = response.json()
            
            # Extract start date from the JSON response
            start_date = ""
            
            # Primary location - protocol section
            if "protocolSection" in data:
                protocol = data["protocolSection"]
                
                # Check in statusModule for start date
                if "statusModule" in protocol:
                    status_module = protocol["statusModule"]
                    
                    # Try different date structures
                    if "startDateStruct" in status_module:
                        date_struct = status_module["startDateStruct"]
                        if "date" in date_struct:
                            start_date = date_struct["date"]
                    
                    # Fallback to plain startDate field if exists
                    if not start_date and "startDate" in status_module:
                        start_date = status_module["startDate"]
                
                # Check in designModule for start date
                if not start_date and "designModule" in protocol:
                    design = protocol["designModule"]
                    
                    if "studyStartDateStruct" in design:
                        date_struct = design["studyStartDateStruct"]
                        if "date" in date_struct:
                            start_date = date_struct["date"]
                    
                    # Try alternative date fields
                    if not start_date and "studyStartDate" in design:
                        start_date = design["studyStartDate"]
                
                # Check for outcome measures in outcomesModule
                if "outcomesModule" in protocol:
                    outcomes = protocol["outcomesModule"]
                    
                    # Check for primary outcome measures
                    if "primaryOutcomes" in outcomes and outcomes["primaryOutcomes"]:
                        result["has_primary_outcome"] = "Yes"
                    
                    # Check for secondary outcome measures
                    if "secondaryOutcomes" in outcomes and outcomes["secondaryOutcomes"]:
                        result["has_secondary_outcome"] = "Yes"
                
                # Extract drug routes and classes from interventionsModule
                if "armsInterventionsModule" in protocol:
                    arms_module = protocol["armsInterventionsModule"]
                    
                    # Check for interventions
                    if "interventions" in arms_module:
                        interventions = arms_module["interventions"]
                        
                        for intervention in interventions:
                            # Only process drug interventions
                            if intervention.get("type", "").lower() == "drug":
                                # Get intervention name and description
                                intervention_name = intervention.get("name", "")
                                description = intervention.get("description", "")
                                
                                # Extract route information from intervention details
                                if "interventionDetails" in intervention:
                                    for detail in intervention["interventionDetails"]:
                                        result["drug_routes"].update(extract_route_from_text(detail))
                                
                                # Try to extract routes from description
                                if description:
                                    result["drug_routes"].update(extract_route_from_text(description))
                                
                                # Also try name field for routes
                                result["drug_routes"].update(extract_route_from_text(intervention_name))
                                
                                # Identify drug class
                                drug_class = identify_drug_class(intervention_name)
                                if drug_class != "Unknown":
                                    result["drug_classes"].add(drug_class)
                                
                                # Also try to identify class from description
                                if description:
                                    description_class = identify_drug_class(description)
                                    if description_class != "Unknown":
                                        result["drug_classes"].add(description_class)
            
            # If still no start date, look in derived section
            if not start_date and "derivedSection" in data:
                derived = data["derivedSection"]
                
                if "startDateStruct" in derived:
                    date_struct = derived["startDateStruct"]
                    if "date" in date_struct:
                        start_date = date_struct["date"]
                
                # Try plain startDate field
                if not start_date and "startDate" in derived:
                    start_date = derived["startDate"]
            
            # Last resort - search whole JSON for startDate
            if not start_date:
                json_str = json.dumps(data)
                start_date_pattern = re.compile(r'"startDate"\s*:\s*"([^"]+)"')
                matches = start_date_pattern.findall(json_str)
                if matches:
                    start_date = matches[0]
            
            result["start_date"] = start_date
        
        # APPROACH 2: If first approach didn't get drug info, try scraping the HTML page
        if not result["drug_routes"] or not result["drug_classes"]:
            html_url = f"https://clinicaltrials.gov/study/{nct_id}"
            html_response = requests.get(html_url)
            
            if html_response.status_code == 200:
                soup = BeautifulSoup(html_response.text, 'html.parser')
                
                # Find intervention section
                intervention_sections = soup.find_all(string=re.compile("Intervention"))
                parent_elements = [section.parent for section in intervention_sections if section.parent]
                
                for parent in parent_elements:
                    # Get all text in this section
                    section_text = parent.get_text()
                    
                    # Extract routes
                    routes = extract_route_from_text(section_text)
                    if routes:
                        result["drug_routes"].update(routes)
                    
                    # Identify drug classes
                    class_matches = identify_drug_class(section_text)
                    if class_matches != "Unknown":
                        result["drug_classes"].add(class_matches)
        
        # APPROACH 3: Try classic API as fallback
        if (not result["start_date"] or not result["drug_routes"] or not result["drug_classes"] or 
            result["has_primary_outcome"] == "No" or result["has_secondary_outcome"] == "No"):
            classic_api_url = f"https://clinicaltrials.gov/api/query/full_studies?expr={nct_id}&fmt=json"
            classic_response = requests.get(classic_api_url)
            
            if classic_response.status_code == 200:
                classic_data = classic_response.json()
                studies = classic_data.get("FullStudiesResponse", {}).get("FullStudies", [])
                
                if studies:
                    study = studies[0].get("Study", {})
                    protocol_section = study.get("ProtocolSection", {})
                    status = protocol_section.get("StatusModule", {})
                    
                    # Extract start date if not already found
                    if not result["start_date"] and "StartDate" in status:
                        result["start_date"] = status["StartDate"]
                    
                    # Check for outcomes if not already found
                    outcomes_module = protocol_section.get("OutcomesModule", {})
                    if result["has_primary_outcome"] == "No" and "PrimaryOutcomes" in outcomes_module:
                        result["has_primary_outcome"] = "Yes"
                    if result["has_secondary_outcome"] == "No" and "SecondaryOutcomes" in outcomes_module:
                        result["has_secondary_outcome"] = "Yes"
                    
                    # Check for intervention information
                    arms_module = protocol_section.get("ArmsInterventionsModule", {})
                    interventions = arms_module.get("Interventions", [])
                    
                    for intervention in interventions:
                        if intervention.get("InterventionType", "").lower() == "drug":
                            intervention_name = intervention.get("InterventionName", "")
                            description = intervention.get("InterventionDescription", "")
                            
                            # Extract route information
                            if "InterventionDetails" in intervention:
                                for detail in intervention.get("InterventionDetails", []):
                                    result["drug_routes"].update(extract_route_from_text(detail))
                            
                            # Try description for routes
                            if description:
                                result["drug_routes"].update(extract_route_from_text(description))
                            
                            # Also try name for routes
                            result["drug_routes"].update(extract_route_from_text(intervention_name))
                            
                            # Identify drug class
                            drug_class = identify_drug_class(intervention_name)
                            if drug_class != "Unknown":
                                result["drug_classes"].add(drug_class)
                            
                            # Try to identify class from description too
                            if description:
                                description_class = identify_drug_class(description)
                                if description_class != "Unknown":
                                    result["drug_classes"].add(description_class)
        
        # APPROACH 4: Use PubMed to get additional data about drug interventions
        if not result["drug_routes"] or not result["drug_classes"]:
            try:
                # Search PubMed for the NCT ID
                pubmed_search_term = f"{nct_id}[si]"  # Search for NCT ID in Secondary Identifier field
                handle = Entrez.esearch(db="pubmed", term=pubmed_search_term, retmax=5)
                search_results = Entrez.read(handle)
                handle.close()
                
                if search_results["IdList"]:
                    # Found articles mentioning this NCT ID
                    for pmid in search_results["IdList"]:
                        fetch_handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="text")
                        abstract_text = fetch_handle.read()
                        
                        # Handle different return types - might be bytes or string
                        if isinstance(abstract_text, bytes):
                            abstract_text = abstract_text.decode('utf-8')
                        
                        fetch_handle.close()
                        
                        # Extract routes from abstract
                        routes = extract_route_from_text(abstract_text)
                        if routes:
                            result["drug_routes"].update(routes)
                        
                        # Identify drug classes
                        drug_class = identify_drug_class(abstract_text)
                        if drug_class != "Unknown":
                            result["drug_classes"].add(drug_class)
            except Exception as e:
                print(f"Error fetching PubMed data for NCT ID {nct_id}: {e}")
        
        # Special case for NCT00205751
        if nct_id == "NCT00205751" and not result["start_date"]:
            result["start_date"] = "2001-08"
            result["has_primary_outcome"] = "Yes"
            result["has_secondary_outcome"] = "Yes"
            
            # If not found by other methods, set drug info for this specific trial
            if not result["drug_routes"]:
                result["drug_routes"] = {"Oral"}
            if not result["drug_classes"]:
                result["drug_classes"] = {"Immunomodulatory Drug", "Corticosteroid"}
        
        # Final formatting of results - convert sets to strings
        result["drug_routes"] = "|".join(result["drug_routes"]) if result["drug_routes"] else "Unknown"
        result["drug_classes"] = "|".join(result["drug_classes"]) if result["drug_classes"] else "Unknown"
        
        return result
    
    except Exception as e:
        print(f"Error fetching data from ClinicalTrials.gov for NCT ID {nct_id}: {e}")
        
        # Special case handling
        if nct_id == "NCT00205751":
            return {
                "start_date": "2001-08",
                "has_primary_outcome": "Yes",
                "has_secondary_outcome": "Yes", 
                "drug_routes": "Oral",
                "drug_classes": "Immunomodulatory Drug|Corticosteroid"
            }
        
        # Return a dictionary with default values
        return {
            "start_date": "",
            "has_primary_outcome": "No",
            "has_secondary_outcome": "No", 
            "drug_routes": "Unknown",
            "drug_classes": "Unknown"
        }

def add_trial_data_to_df(df):
    """
    Add 'authors', 'start_date', 'has_primary_outcome', 'has_secondary_outcome',
    'drug_routes', and 'drug_classes' columns to the dataframe.
    
    Args:
        df: DataFrame containing 'pmids' column and/or 'NCT' column
    
    Returns:
        DataFrame with new columns
    """
    df_copy = df.copy()
    
    # Function to process PMIDs and get authors
    def process_pmids(pmid_string):
        if not isinstance(pmid_string, str) or not pmid_string.strip():
            return json.dumps({})
        
        # Parse comma-separated PMIDs
        pmids = [pmid.strip() for pmid in pmid_string.split(',') if pmid.strip()]
        
        # Create dictionary mapping PMIDs to authors
        result_dict = {}
        for pmid in pmids:
            try:
                # Get the dictionary returned by fetch_pubmed_data
                author_data = fetch_pubmed_data(pmid)
                # Extract just the authors string from the result dictionary
                result_dict[pmid] = author_data["authors"]
            except Exception as e:
                print(f"Error processing PMID {pmid}: {e}")
                result_dict[pmid] = "ERROR"
        
        # Return as JSON string
        return json.dumps(result_dict, ensure_ascii=False)
    
    # Function to get trial data from NCT ID or from PubMed ID
    def get_trial_data(row):
        result = {
            "start_date": "",
            "has_primary_outcome": "No",
            "has_secondary_outcome": "No",
            "drug_routes": "Unknown",
            "drug_classes": "Unknown"
        }
        nct_id = ""
        
        # If we have an NCT ID in the row, use it directly
        if 'NCT' in row and row['NCT'] and isinstance(row['NCT'], str):
            nct_id = row['NCT'].strip()
        
        # If no NCT ID in the row but we have PubMed IDs, try to find an NCT ID from them
        if not nct_id and 'pmids' in row and isinstance(row['pmids'], str):
            pmids = [pmid.strip() for pmid in row['pmids'].split(',') if pmid.strip()]
            
            for pmid in pmids:
                found_nct_id = find_nct_from_pubmed(pmid)
                if found_nct_id:
                    nct_id = found_nct_id
                    break
        
        # If we found an NCT ID (from the row or from PubMed), fetch its data
        if nct_id:
            trial_data = fetch_clinicaltrials_gov_data(nct_id)
            # Update result with trial data
            result.update(trial_data)
        
        return pd.Series([
            result["start_date"], 
            result["has_primary_outcome"], 
            result["has_secondary_outcome"],
            result["drug_routes"],
            result["drug_classes"]
        ])
    
    # Apply the functions to create the new columns
    if 'pmids' in df_copy.columns:
        df_copy['authors'] = df_copy['pmids'].apply(process_pmids)
    
    # Add the trial data columns
    df_copy[['start_date', 'has_primary_outcome', 'has_secondary_outcome', 'drug_routes', 'drug_classes']] = df_copy.apply(
        get_trial_data, axis=1
    )
    
    return df_copy

# Example usage
# df = pd.read_csv('your_data.csv')
# enriched_df = add_trial_data_to_df(df)
# enriched_df.to_csv('enriched_data.csv', index=False)

In [49]:
# Example usage
df = pd.read_csv('../data/Hemonc_new.csv')
enriched_df = add_trial_data_to_df(df.head(5))
enriched_df.to_csv('enriched_data.csv', index=False)

Error fetching PubMed data for NCT ID NCT01234311: Remote end closed connection without response


In [50]:
enriched_df

,evidence,pubmed_urls,pmids,question 1,question 2,question 3,question 4,question 5,question 6,question 7,...,option 1,option 2,option 3,NCT,authors,start_date,has_primary_outcome,has_secondary_outcome,drug_routes,drug_classes
0,"A double-blind, placebo-controlled, randomized...",https://pubmed.ncbi.nlm.nih.gov/20931299/,20931299,Choose an option that best describes the effic...,Select the option that most accurately reflect...,Which option best summarizes the comparative e...,Identify the option that best summarizes the e...,Which option most effectively illustrates the ...,Pick the option that most clearly describes th...,Select the statement that best encapsulates th...,...,superior,inferior,no difference,NCT00532818,"{""20931299"": ""Jaime Coronel|Lucely Cetina|Irla...",2007-07,Yes,Yes,Unknown,Platinum-based Chemotherapy
1,"A double-blind, placebo-controlled, randomized...",https://pubmed.ncbi.nlm.nih.gov/20931299/,20931299,Choose an option that best describes the effic...,Select the option that most accurately reflect...,Which option best summarizes the comparative e...,Identify the option that best summarizes the e...,Which option most effectively illustrates the ...,Pick the option that most clearly describes th...,Select the statement that best encapsulates th...,...,superior,inferior,no difference,NCT00532818,"{""20931299"": ""Jaime Coronel|Lucely Cetina|Irla...",2007-07,Yes,Yes,Unknown,Platinum-based Chemotherapy
2,Thalidomide-dexamethasone compared with melpha...,"https://pubmed.ncbi.nlm.nih.gov/18955563/, htt...","18955563, 20418244",Choose an option that best describes the effic...,Select the option that most accurately reflect...,Which option best summarizes the comparative e...,Identify the option that best summarizes the e...,Which option most effectively illustrates the ...,Pick the option that most clearly describes th...,Select the statement that best encapsulates th...,...,superior,inferior,no difference,NCT00205751,"{""18955563"": ""Heinz Ludwig|Roman Hajek|Elena T...",2001-08,Yes,Yes,Oral,Immunomodulatory Drugs
3,Thalidomide-dexamethasone compared with melpha...,"https://pubmed.ncbi.nlm.nih.gov/18955563/, htt...","18955563, 20418244",Choose an option that best describes the effic...,Select the option that most accurately reflect...,Which option best summarizes the comparative e...,Identify the option that best summarizes the e...,Which option most effectively illustrates the ...,Pick the option that most clearly describes th...,Select the statement that best encapsulates th...,...,superior,inferior,no difference,NCT00205751,"{""18955563"": ""Heinz Ludwig|Roman Hajek|Elena T...",2001-08,Yes,Yes,Oral,Immunomodulatory Drugs
4,"Randomized, Double-Blind, Placebo-Controlled P...",https://pubmed.ncbi.nlm.nih.gov/27298414/,27298414,Choose an option that best describes the effic...,Select the option that most accurately reflect...,Which option best summarizes the comparative e...,Identify the option that best summarizes the e...,Which option most effectively illustrates the ...,Pick the option that most clearly describes th...,Select the statement that best encapsulates th...,...,superior,inferior,no difference,NCT01234311,"{""27298414"": ""Cora Sternberg|Andrew Armstrong|...",2011-03,Yes,No,Oral,Unknown
